# 승리 전략 도출을 위한 상세 데이터 분석

<목차>

1. 개요

2. EDA

3. 데이터 분석

4. 분석 결론

---
---

## 개요

### 1. 목적
- 승리에 주요하게 영향을 미치는 특성과 내 경기의 특성을 비교 분석하여 <월드클래스>로 승급하기 위한 FC온라인 경기 승리 전략을 수립한다.

### 2. 데이터 설명
1. FC온라인 API를 통해 수집한 다른 유저들의 경기 데이터
    - 득점한 선수에 관련한 데이터는 분석에 활용하지 않고, 유저의 플레이 데이터만 활용한다.
    - 매치ID별, 유저별 경기 기록 데이터

2. 머신러닝에 활용한 데이터
    - 전처리 완료한 유저별 경기 기록 데이터

### 3. 분석 환경 조성

#### ㄴ 라이브러리 호출

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ▶ Warnings 제거
import warnings
warnings.filterwarnings('ignore')

# ▶ 한글 폰트 설정하기
plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] =False

#### ㄴ 데이터 호출

In [6]:
# 데이터 프레임 생성
match_df = pd.read_csv('./data/match_data.csv', encoding='utf-8-sig')
match_merge_df = pd.read_csv('./data/match_merge_data.csv', encoding='utf-8-sig')
outlier_processed_df = pd.read_csv('./data/outlier_processed_data.csv', encoding='utf-8-sig')

print(match_df.shape)
print(match_merge_df.shape)
print(outlier_processed_df.shape)

(1931, 43)
(270, 30)
(270, 29)


In [7]:
# 데이터 프레임별 head 확인
match_df.head()

,matchId,matchDate,matchType,ouid,nickname,matchResult,matchEndType,foul,injury,redCards,...,goalTime,x,y,type,result,spId,assist,assistSpId,hitPost,inPenalty
0,67e5abd6293b2e29d79b74f4,2025-03-27T19:59:51,50,78bd4904e360e8f15281c765ad9069e1,김진효,무,0,0,0,0,...,1545,0.936611,0.543738,2,3,827167198,True,825054050,False,True
1,67e5abd6293b2e29d79b74f4,2025-03-27T19:59:51,50,78bd4904e360e8f15281c765ad9069e1,김진효,무,0,0,0,0,...,16777585,0.846402,0.387678,1,1,827167198,True,814200145,False,True
2,67e5abd6293b2e29d79b74f4,2025-03-27T19:59:51,50,78bd4904e360e8f15281c765ad9069e1,김진효,무,0,0,0,0,...,16777916,0.818322,0.401577,2,2,827167198,True,814200145,False,False
3,67e5abd6293b2e29d79b74f4,2025-03-27T19:59:51,50,78bd4904e360e8f15281c765ad9069e1,김진효,무,0,0,0,0,...,16779340,0.887430,0.428050,2,2,827167198,True,825054050,False,True
4,67e5abd6293b2e29d79b74f4,2025-03-27T19:59:51,50,78bd4904e360e8f15281c765ad9069e1,김진효,무,0,0,0,0,...,16779891,0.905626,0.629533,6,1,825054050,True,252020801,False,True


In [8]:
# 데이터 프레임별 head 확인
match_merge_df.head()

,matchId,ouid,foul,injury,redCards,yellowCards,dribble,cornerKick,offsideCount,shootTotal,...,throughPassTry,throughPassSuccess,tackleTry,tackleSuccess,blockTry,blockSuccess,possession,averageRating,matchDate,matchResult
0,67e5ab6c26956677381da20f,00bb68e0fa9a9f7cb1665b32dd8069b7,1.0,0.0,0.0,0.0,64.0,3.0,0.0,3.0,...,16.0,16.0,17.0,9.0,5.0,1.0,44.0,3.63333,2025-03-27T19:59:05,패
1,67e5ab6c26956677381da20f,b732daf0b96fc9256262fb36cb72c1bc,0.0,0.0,0.0,0.0,75.0,1.0,1.0,14.0,...,26.0,25.0,13.0,8.0,5.0,0.0,56.0,4.59444,2025-03-27T19:59:05,승
2,67e5ab6d0b020281e49f1496,7fe7ffb96f2154e14825d674841ff270,0.0,0.0,0.0,0.0,80.0,2.0,0.0,7.0,...,19.0,17.0,6.0,4.0,8.0,1.0,58.0,4.21667,2025-03-27T19:58:36,승
3,67e5ab6d0b020281e49f1496,a184d3be0a42e0978644eb62b3b41ff5,1.0,0.0,0.0,0.0,68.0,1.0,0.0,8.0,...,22.0,18.0,21.0,9.0,6.0,0.0,42.0,3.82778,2025-03-27T19:58:36,패
4,67e5ab6edc4cb4e86e4f9236,eb36555d4d460edc57ee20b782723388,0.0,0.0,0.0,0.0,72.0,0.0,1.0,6.0,...,15.0,13.0,4.0,1.0,15.0,2.0,44.0,4.07222,2025-03-27T19:59:02,패


In [9]:
# 데이터 프레임별 head 확인
outlier_processed_df.head()

,ouid,foul,injury,redCards,yellowCards,dribble,cornerKick,offsideCount,shootTotal,effectiveShootTotal,...,throughPassTry,throughPassSuccess,tackleTry,tackleSuccess,blockTry,blockSuccess,possession,averageRating,matchResult,hour
0,00bb68e0fa9a9f7cb1665b32dd8069b7,1.0,0.0,0.0,0.0,64.0,3.0,0.0,3.0,2.0,...,16.0,16.0,17.0,9.0,5.0,1.0,44.0,3.63333,패,19
1,b732daf0b96fc9256262fb36cb72c1bc,0.0,0.0,0.0,0.0,75.0,1.0,1.0,14.0,9.0,...,26.0,25.0,13.0,8.0,5.0,0.0,56.0,4.59444,승,19
2,7fe7ffb96f2154e14825d674841ff270,0.0,0.0,0.0,0.0,80.0,2.0,0.0,7.0,6.0,...,19.0,17.0,6.0,4.0,8.0,1.0,58.0,4.21667,승,19
3,a184d3be0a42e0978644eb62b3b41ff5,1.0,0.0,0.0,0.0,68.0,1.0,0.0,8.0,7.0,...,22.0,18.0,21.0,9.0,6.0,0.0,42.0,3.82778,패,19
4,eb36555d4d460edc57ee20b782723388,0.0,0.0,0.0,0.0,72.0,0.0,1.0,6.0,4.0,...,15.0,13.0,4.0,1.0,15.0,2.0,44.0,4.07222,패,19


---

## EDA
1. 골

2. 점유율

3. 슛

4. 패스